In [ ]:
#| default_exp markdown.obsidian.personal.notation

# markdown.obisidian.personal.notation
> Functions for making and managing notation cards

Mathematical texts are difficult to read not only because it introduces readers to new concepts that they are unfamiliar with but also because any given excerpt requires the reader to be familiar with notation that the writer chooses to use.

The first time reader of a mathematical text thus must go on a wild goose hunt to find where the unfamiliar notations are defined. Unfortunately, the definitions may further introduce unfamiliar notations.

Notation notes try to ease this problem to an extent by providing
1. links to quickly find where notations are introduced
2. the contexts under which the notations are defined, and
3. miscellaneous descriptions of the notations as necessary. 

In [ ]:
#| export
from os import PathLike
from pathlib import Path
import re
from typing import Optional, Union
import warnings

from multiset import Multiset
from pylatexenc.latexwalker import LatexNode, LatexMacroNode, LatexWalker, LatexGroupNode, LatexCharsNode

from trouver.helper import notation_asterisk_indices
from trouver.markdown.markdown.file import MarkdownFile, MarkdownLineEnum
from trouver.markdown.obsidian.links import (
    find_links_in_markdown_text, LinkType, ObsidianLink, MARKDOWNLINK_PATTERN, WIKILINK_PATTERN
)
from trouver.markdown.obsidian.personal.information_notes import bulleted_links_of_type_in_section
from trouver.markdown.obsidian.personal.note_type import (
    PersonalNoteTypeEnum, assert_note_is_of_type, note_is_of_type
)
from trouver.markdown.obsidian.vault import VaultNote, NotePathIsNotIdentifiedError, NoteDoesNotExistError

In [ ]:
import os
from unittest import mock
import shutil
import tempfile

from fastcore.test import *
from pathvalidate import validate_filename 

from trouver.helper import _test_directory

## Getting a path-valid string from a string containing latex

In [ ]:
#| export
CHARACTER_ORDERING_LIST =\
    ['A', 'a', r'\Alpha', r'\alpha', 'B', 'b', r'\Beta', r'\beta', 'C', 'c', r'\Gamma',
     r'\gamma', 'D', 'd', r'\Delta', r'\delta', 'E', 'e', r'\Epsilon', r'\epsilon',
     'F', 'f', 'G', 'g', 'H', 'h', r'\Eta', r'\eta', 'I', 'i', r'\Iota', r'\iota',
     'J', 'j', 'K', 'k', r'\Kappa', r'\kappa', 'L', 'l', r'\Lambda', r'\lambda', 'M',
     'm', r'\Mu', r'\mu', 'N', 'n', r'\Nu', r'\nu', 'O', 'o', r'\Omicron', r'\omicron'
     'P', 'p', r'\Pi', r'\pi', r'\Phi', r'\phi', r'\Psi', r'\psi', 'Q', 'q', 'R', 'r', 
     r'\Rho', r'\rho', 'S', 's', r'\Sigma', r'\sigma', 'T', 't', r'\Theta', r'\theta',
     r'\Tau', r'\tau', 'U', 'u', r'\Upsilon', r'\upsilon', 'V', 'v', 'W', 'w', r'\Omega', r'\omega',
     'X', 'x', '\Chi', '\chi', 'Y', 'y', 'Z', 'z', '\Zeta', '\zeta', '*', r'\bullet']
DECORATING_CHARACTERS =\
    [r'\tilde', r'\hat', r'\overline', r'\bar', r'\mathscr', r'\mathcal',
     r'\mathfrak', r'\\operatorname', r'\\text', r'\\bf']
NONEFFECTIVE_CHARACTERS =\
    ['^', '_', '{', '}', '(', ')', '[', ']']

In [ ]:
#| export
TO_REMOVE = [
    '.', '\'', '$', ')', '{', '}', ':', '?', '!', '#', '%', '&',
    '\\', '<', '>', '*', '?', '"', '@', '+', '`', '|', '=', '[', ']',
    'mathscr', 'mathbf', 'mathrm', 'mathfrak', 'mathcal', 'mathbb', 'operatorname',
    'left', 'right']
TO_UNDERSCORE = [' ', '-', '^', '(', ',', '/']

# TODO: make a universal latex to path string; it seems that latex.convert
# might do something different when naming files.

def latex_to_path_accepted_string(latex: str) -> str:
    """Convert a latex string to a path accepted string
    """
    for to_remove in TO_REMOVE:
        latex, _ = re.subn(re.escape(to_remove), '', latex)
    for to_underscore in TO_UNDERSCORE:
        latex, _ = re.subn(re.escape(to_underscore), '_', latex)
    return latex

We can turn a latex str into a path-valid string.

In [ ]:
sample_1 = r'\mathcal{O}_X'
output_1 = latex_to_path_accepted_string(sample_1)
print(output_1)
assert 'O' in output_1 and 'X' in output_1
validate_filename(output_1)

sample_2 = r'\operatorname{Gal}(L/K)'
output_2 = latex_to_path_accepted_string(sample_2)
print(output_2)
assert 'Gal' in output_2 and 'L' in output_2 and 'K' in output_2
validate_filename(output_2)

O_X
Gal_L_K


# Parse notation note

For the purposes of `trouver`, a notation note most usually starts in the format `<Notation> [[<link_to_note>|denotes]] <explanation of what that notation denotes/is defined as> <optional paragraphs discussing aspects about the notation>`. For example, "$\deg D$ [[note_link|denotes]] the degree of the divisor $D$" would be an example of such a note.

If the word `denotes` is not given in a link to a note, then the note to which the first link points is considered the main note of the notation note. Alternatively, a notation note might also sometimes have a comment citing the source of the notation instead of links.  Nevertheless, it is preferred that `denotes` is given in a link.
 
A notation note may have YAML frontmatter meta as well.


In [ ]:
#| export
def _main_of_notation_from_text(
        file_text: str # Text of notation note
        ) -> Union[str, VaultNote, None]: # The name main information note that `notation_note` comes from. Returns `None` if `notation_note` does not come from such a note.
    """Return the name of the note from which the notation comes from.

    Helper function for `parse_notation_string`.
    """
    if '%%' in file_text and 'main: ' in file_text:
        return None

    # First, get rid of the notation, i.e. the first latex math mode string
    match = re.search(fr'\$.+?\$\s+', file_text) 
    if match is None:
        raise ValueError(
            'There seems to be a formatting error in a notation note'
            ' and the notation has not been identified. The following is the'
            f' name of the notation note: {notation_note.name}')
    start, end = match.span()
    file_text = file_text[end:]
    
    link_locations = find_links_in_markdown_text(file_text)
    if not link_locations:
        return None
    start, end = link_locations[0]
    link_str = file_text[start:end]
    link_parse = ObsidianLink.from_text(link_str)
    main_note_name = link_parse.file_name
    return main_note_name

In [ ]:
#| hide
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A')
test_eq(_main_of_notation_from_text(notation_note.text()), 'spectrum_of_a_ring')

In [ ]:
#| hide
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_O_X_this_file_has_no_links')
main_note =  _main_of_notation_from_text(notation_note.text())
assert main_note is None

In [ ]:
#| hide
# Test the case where the notation string contains a [[]]
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_k_t_formal_power_series_ring') 
_main_of_notation_from_text(notation_note.text())


'some_note'

In [ ]:
#| export
def _divide_bulleted_list_mf_at_end(
        mf: MarkdownFile
        ) -> tuple[MarkdownFile, Union[MarkdownFile, None]]: # The first MarkdownFile contains the main content. The second MarkdonwFile contains the bulleted list at the end; if no such bulleted list exists, then this is None.
    """Divide a `MarkdownFile` for a notation note into two MarkdownFiles, one
    of the main content and the other for the trailing bulleted list of links
    for notations used in the notation note.

    Assumes that the bulleted list is formatted correctly
    (i.e. each line is of the form `- [<notation>](<link>)`)

    Helper function for `parse_notation_note`.
    """
    main_parts = mf.parts.copy()
    trailing_parts = []
    for part in reversed(mf.parts):
        if part['type'] == MarkdownLineEnum.BLANK_LINE:
            main_parts.pop()
            continue
        if not _part_is_unordered_list_and_is_of_markdownstyle_link(part):
            break
        last_part = main_parts.pop() # Should be the same as `part`
        trailing_parts.insert(0, last_part)
    
    if trailing_parts:
        bulleted_list_mf = MarkdownFile(trailing_parts)
    else:
        bulleted_list_mf = None
    return MarkdownFile(main_parts), bulleted_list_mf

def _part_is_unordered_list_and_is_of_markdownstyle_link(
        part: dict[str, Union[str, MarkdownLineEnum]]
        ) -> bool:
    """
    
    Helper function for `_divide_bulleted_list_mf_at_end`
    """
    if part['type'] != MarkdownLineEnum.UNORDERED_LIST:
        return False
    if not part['line'].startswith('- '):
        return False
    if not re.match(MARKDOWNLINK_PATTERN, part['line'][2:]):
        return False
    return True
    

In [ ]:
#| hide
notation_note_str = '$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.\n- [$L$](notation_L_some_field_extension)\n- [$K$](notation_K_some_base_field)'
mf = MarkdownFile.from_string(notation_note_str)
main_content, bulleted_list = _divide_bulleted_list_mf_at_end(mf)
test_eq(str(main_content), '$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.')
test_eq(str(bulleted_list), '- [$L$](notation_L_some_field_extension)\n- [$K$](notation_K_some_base_field)')

# This time, add more blank lines
notation_note_str = '$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.\n\n- [$L$](notation_L_some_field_extension)\n\n- [$K$](notation_K_some_base_field)\n'
mf = MarkdownFile.from_string(notation_note_str)
main_content, bulleted_list = _divide_bulleted_list_mf_at_end(mf)
test_eq(str(main_content), '$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.')
test_eq(str(bulleted_list), '- [$L$](notation_L_some_field_extension)\n- [$K$](notation_K_some_base_field)')

# No bulleted list means the second output is `None`:

notation_note_str = '$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.\n\n'
mf = MarkdownFile.from_string(notation_note_str)
main_content, bulleted_list = _divide_bulleted_list_mf_at_end(mf)
test_eq(str(main_content), '$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.')
assert bulleted_list is None



In [ ]:
#| export
def parse_notation_note(
        notation_note: Union[str, VaultNote],
        vault: Optional[PathLike] = None # The vault If `None`, then uses th
        ) -> tuple[Union[dict, None], Union[str, None], str, MarkdownFile,
                   Union[MarkdownFile, None]]:
    """Parse information from the notation note.

    **Returns**

    - tuple[Union[dict, None], str, ObsidianLink, MarkdownFile, MarkdownFile]
        - The first entry is the YAML frontmatter meta, if available.
        - The second entry is the notation string
        - The third entry is the name of the "main note" of the notation note. This is usual
          the linked note in the link `[[<linked_note>|denotes]]`. If no such main note
          exists, then this is `None`.
        - The fourth entry is the MarkdownFile consisting of the "main" content of the note,
          which excludes the information given by all of the other entries.
        - The fifth entry is the MarkdownFile consisting of the ending bulleted list, listing
          the notations used in the notation notes along with links to the notation notes
          describing these notations. If there is not such bulleted list, then this entry
          is `None`. 

    **Raises**

    - UserWarning
        - If the (non-YAML frontmatter meta) contents of the note do not start
        inn the form `<Notation> [[<link_to_note>|denotes]]`; the name of the
        notation note is included in the warning message.
    - ValueError
        - If the notation note is not formatted correctly by starting
        with the notation with dollar signs `$`.
    - AssertionError
        - If `notation_note` is not determined to be a notation note.
    """
    if isinstance(notation_note, str):
        notation_note = VaultNote(vault, name=notation_note)
    if not vault:
        vault = notation_note.vault
    assert_note_is_of_type(notation_note, PersonalNoteTypeEnum.NOTATION_NOTE)

    mf = MarkdownFile.from_vault_note(notation_note)
    metadata = mf.metadata()
    mf_without_metadata = MarkdownFile(
        [part for part in mf.parts if part['type'] != MarkdownLineEnum.META])

    file_text = str(mf_without_metadata)

    main_mf, mf_with_links_to_notations = _divide_bulleted_list_mf_at_end(mf_without_metadata)
    _remove_the_notation_str_and_denotes_in_main_mf(main_mf, notation_note)

    return (metadata, _get_notation_string(file_text, notation_note),
            _main_of_notation_from_text(file_text), main_mf,
            mf_with_links_to_notations)


def _get_notation_string(
        file_text: str,
        notation_note: VaultNote
        ) -> str:
    """Return the notation string from the text of the notation note..

    Assumes that the notation string exists and is well formatted.

    Helper function for `parse_notation_note`.
    """
    try:
        return re.search(r'\$.+?\$', file_text).group()
    except AttributeError as e:
        raise ValueError(
            'There seems to be a formatting error in a notation note'
            ' and the notation has not been identified. The following is the'
            f' name of the notation note: {notation_note.name}')
    

def _remove_the_notation_str_and_denotes_in_main_mf(
        main_mf: MarkdownFile,
        notation_note: VaultNote):
    """Remove the text `<notation> denotes ` which starts the
    notation description.

    Helper function of `parse_notation_note`.
    """
    for part in main_mf.parts:
        if part['type'] == MarkdownLineEnum.BLANK_LINE:
            continue
        match = re.match(fr'^\$.+?\$ ({WIKILINK_PATTERN}|denotes)\s*', part['line']) 
        if match is None:
            raise ValueError(
                'There seems to be a formatting error in a notation note'
                ' and the notation has not been identified. The following is the'
                f' name of the notation note: {notation_note.name}')
        else:
            start, end = match.span()
            part['line'] = part['line'][end:]
            break
    

`parse_notation_note` gets information about the notation note. Note that the `MarkdownFile` object `main_mf` that has the main content/description of the notation does not start with the pharse of the form `<notation> [[<link>|denotes]] `.

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A')
metadata, notation_str, main_of_notation, main_mf, mf_with_links_to_notations = parse_notation_note(notation_note, vault)

test_eq(metadata, {'detect_regex': [], 'latex_in_original': ['\\operatorname{Spec} A']})
test_eq(notation_str, '$\\operatorname{Spec} A$')
test_eq(main_of_notation, 'spectrum_of_a_ring')
test_eq(str(main_mf), 'the spectrum of the ring $A$.')
assert mf_with_links_to_notations is None # There is not a bulleted list at the end, so the last output is `None`.

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='poonen_curves_notation_zeta_X_s_zeta_function_of_variety')
metadata, notation_str, main_of_notation, main_mf, mf_with_links_to_notations = parse_notation_note(notation_note, vault)

test_eq(metadata, None)
test_eq(notation_str, r'$\zeta_{X}(s)$')
test_eq(main_of_notation, 'poonen_curves_3.4.1 DEFINITION')
test_eq(str(main_mf), 'the zeta function of the [[poonen_curves_1.0.2 DEFINITION|variety]] $X$ over $\\mathbb{F}_q$.\n\nIt is defined as\n\n$$\\zeta_X(s) = Z_X(q^{-s}).$$\n\nA priori, it is a formal series, but in fact [[poonen_curves_ 3.6_page_56|it converges]] for $\\operatorname{Re} s > \\dim X$.')
test_eq(str(mf_with_links_to_notations), '- [$Z_X$](poonen_curves_notation_Z_X_T)') # There is a bulleted list at the end, so the last output is `None`.

In [ ]:
#| hide
# Test the case where the note has no main note; in particular, it starts with `<notation> denotes ` with no link!`

vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_O_X_this_file_has_no_links') 
metadata, notation_str, main_of_notation, main_mf, mf_with_links_to_notations = parse_notation_note(notation_note, vault)
assert main_note is None


In [ ]:
#| hide
# Test the case where the note not blank lines after the `denotes`. 

vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_B_R') 
metadata, notation_str, main_of_notation, main_mf, mf_with_links_to_notations = parse_notation_note(notation_note, vault)
assert main_note is None


In [ ]:
#| hide
# Test the case where the notation string contains a [[]]
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_k_t_formal_power_series_ring') 
metadata, notation_str, main_of_notation, main_mf, mf_with_links_to_notations = parse_notation_note(notation_note, vault)
test_eq(notation_str, '$k[[t]]$')
test_eq(main_of_notation, 'some_note')
# assert main_note is None

In [ ]:
#| export
def notation_in_note(
        notation_note: Union[str, VaultNote],
        vault: Optional[PathLike] = None 
        ) -> str:
    """Return the name of the note from which the notation comes from.
    
    **Parameters**

    - `notation_note` - Union[str, VaultNote]
        - Either
            
            - The name of the notation note or
            - The `VaultNote` object of the notation note. 
            
        The note name is expected to be unique
        inside the vault specified by `vault`. 
        This is expected to contain `'notation'` as a substring. 
        Usually, this is expected to be formatted in one of
        the following forms:
            - `'<reference_name>_notation_<rest_of_note_name>'`
            - `'notation.<rest_of_note_name>'
    - `vault` - Pathlike or `None`
        - Defaults to `None`
        
    **Returns**

    - str
        - The notation in LaTeX, including the dollar signs `$`.

    **Raises**

    """
    _, notation_in_note, _, _, _ = parse_notation_note(notation_note, vault)
    return notation_in_note

`notation_in_note` identifies the notation LaTeX str that a notation note presents. Its output starts and ends with dollar signs.

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A')
notation = notation_in_note(notation_note)
assert notation == r'$\operatorname{Spec} A$'

In [ ]:
#| export
def main_of_notation(
        notation_note: VaultNote, # The VaultNote object representing the notation note.
        as_note: bool = False # If `False`, then returns the name of the note, and returns a VaultNote object with the same vault as `notation_note` otherwise. The vault used to get the `VaultNote` is the vault of `notation_note`.
        ) -> Union[str, VaultNote, None]: # The (name of the) main information note that `notation_note` comes from. Returns `None` if `notation_note` does not come from such a note.
    """Return the name of the note from which the notation comes from.
            
    **Raises**

    - ValueError
        - If the notation note is not formatted correctly by starting
        with the notation with dollar signs `$`.
    """
    vault = notation_note.vault
    _, _, main_note_name, _, _ = parse_notation_note(notation_note, vault)
    if main_note_name is None:
        return None
    if as_note:
        return VaultNote(notation_note.vault, name=main_note_name)
    else:
        return main_note_name

We can identify the "main note" of a notation note with the `main_of_notation` method:

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A')
test_eq(main_of_notation(notation_note), 'spectrum_of_a_ring')

We can also return this main note as a `VaultNote` object:

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A')
main_note =  main_of_notation(notation_note, as_note=True)
assert isinstance(main_note, VaultNote)
test_eq(main_note.name, 'spectrum_of_a_ring')

If the notation note has no links, then `main_of_notation` returns `None`:

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_O_X_this_file_has_no_links')
main_note =  main_of_notation(notation_note)
assert main_note is None

main_note = main_of_notation(notation_note, as_note=True)
assert main_note is None

# Notations in a standard information note

## Find notations introduced in note

In [ ]:
#| export
def notation_str_in_a_standard_information_note(
        info_note: VaultNote
        ) -> list[str]: # Each str is a LaTeX str, beginning and trailing dollar signs `$` (single or double) included.
    """
    Return the LaTeX str's with notations in a standard information note.

    A LaTeX str is deemed to be a notation if it is surrounded by double
    asterisks `**`
    """
    mf = MarkdownFile.from_vault_note(info_note)
    notations = []
    for part in mf.parts:
        indices = notation_asterisk_indices(part['line'])
        notations.extend([
            part['line'][start+2:end-2] for start, end in indices])
    return notations

We can obtain the notation str in an information note:

In [ ]:
vault = _test_directory() / 'test_vault_7'
# A note with just one notation:
vn = VaultNote(vault, name='galois_group')
sample_output_1 = notation_str_in_a_standard_information_note(vn)
assert len(sample_output_1) == 1
assert sample_output_1[0].startswith('$')
assert sample_output_1[0].endswith('$')
assert not sample_output_1[0].startswith('$$')
assert not sample_output_1[0].endswith('$$')
print(sample_output_1)

# A note with a notation with double asterisks:
vn = VaultNote(vault, name='spectrum_of_a_ring')
sample_output_2 = notation_str_in_a_standard_information_note(vn)
assert len(sample_output_2) == 2
assert sample_output_2[1].startswith('$$')
assert sample_output_2[1].endswith('$$')
print(sample_output_2)

# A note with no notations:
assert not notation_str_in_a_standard_information_note(VaultNote(vault, name='no_notations_note_about_integral_domains'))

['$\\operatorname{Gal}(L/K)$']
['$\\operatorname{Spec} A$', '$$D(f) = \\{\\mathfrak{p} \\in \\operatorname{Spec} A: f \\not\\in \\mathfrak{p} \\}.$$']


In [ ]:
# hide
# Test notation str with asterisks.
vault = _test_directory() / 'test_vault_7'
vn = VaultNote(vault, name='direct_and_inverse_images_of_sheaves')
sample_output = notation_str_in_a_standard_information_note(vn)
# sample_output
assert len(sample_output) == 2

## Notation notes listed in see also section

In [ ]:
#| export
def notation_notes_linked_in_see_also_section(
        info_note: VaultNote,
        vault: PathLike, # Path to the vault directory.
        as_vault_notes: bool = True # If `True`, returns each notation note as a `VaultNote` object.  Otherwise, returns the name of each notation note. Defaults to `True`.
        ) -> Union[list[VaultNote], list[str]]: # Each entry corresponds to a notation note in the vault.
    """Return a list of notation notes listed in the
    `See Also` section of the standard information note.

    In the current implementation of this function, only 
    "notation notes" that actually exist are included in
    the returned list.
    """
    links = bulleted_links_of_type_in_section(
        info_note, vault, section="See Also",
        note_type=PersonalNoteTypeEnum.NOTATION_NOTE)
    note_names = [link.file_name for link in links]
    if as_vault_notes:
        return [VaultNote(vault, name=note_name) for note_name in note_names]
    else:
        return note_names


The `notation_notes_linked_in_see_also_section` method detects the Notation notes listed in bulleted links in the See Also section of a standard information note.

In [ ]:
vault = _test_directory() / 'test_vault_7'
vn = VaultNote(vault, name='twist_of_a_graded_module')
sample_output = notation_notes_linked_in_see_also_section(vn, vault)
assert len(sample_output) == 1
assert isinstance(sample_output[0], VaultNote)
print(sample_output[0].name)
assert sample_output[0].exists()

foag_notation_M_n_bullet


Setting `as_vault_notes=False` returns the names of the notation notes.

In [ ]:
vault = _test_directory() / 'test_vault_7'
vn = VaultNote(vault, name='twist_of_a_graded_module')
sample_output = notation_notes_linked_in_see_also_section(vn, vault, as_vault_notes=False)
assert len(sample_output) == 1
assert isinstance(sample_output[0], str)
print(sample_output[0])

foag_notation_M_n_bullet


If a "notation note" does not exist, then it is not included in the returned list.

Note that `notation_notes_linked_in_see_also_section` uses `bulleted_links_of_type_in_section`, which is turn uses `note_is_of_type` to get a list of linked notes of the type `NOTATION_NOTE` of the `PersonalNoteTypeEnum` class. In turn, a note file must exist for the note to be considered of any particular type under the current implementation of `note_is_of_type`.

In [ ]:
vault = _test_directory() / 'test_vault_7'
vn = VaultNote(vault, name='note_with_links_to_non_existent_notation_notes')
sample_output = notation_notes_linked_in_see_also_section(vn, vault, as_vault_notes=True)
test_eq(sample_output, [])
sample_output = notation_notes_linked_in_see_also_section(vn, vault, as_vault_notes=False)
test_eq(sample_output, [])

## Find all notation notes in a vault subdirectory

In [ ]:
#| export 
def notations_and_main_notes(
        vault: PathLike, # Path to the vault directory.
        subdirectory: Optional[PathLike] = None, # Path to the subdirectory, relative to `vault`, to find the notation notes. Searches for all notation notes here and in subdirectories of this subdirectory. If `None`, then the `note parameter is used to determined the subdirectory. If `subdirectory` is the empty str, then all notation notes in the vault are searched. Defaults to `None`. 
        note: Optional[VaultNote] = None # A note in the vault. The directory that this note is in determines the `subdirectory` parameter if the argument passed to `subdirectory` is the blank str. This note can usually be an index note, e.g. `'_index_silverman'`. Defaults to `None`, in which case `subdirectory` must be specified.
        ) -> dict[str, Union[str, None]]: # A key is the unique name of a notation note in the vault and its corresponding value is the name of the main note of the notation note. Each main note may not actually exist, but each notation note definitely exists. If the notation note has no main note (i.e. has no links to other notes), then the value is `None`.
    """Return a `dict` with all of notation notes in a specified
    subdirectory of a vault and the names of the main notes of these
    notation notes.
    
    **Returns**
    - `dict`

    **Raises**

    - ValueError
        - If `subdirectory` and `note` are both `None`.
    """
    if note is None and subdirectory is None:
        raise ValueError(
            'Both the `subdirectory` and `note` parameters are None.')
    # vault = vault if vault is not None else ''
    if subdirectory is None:
        subdirectory = Path(note.rel_path).parent
    subdirectory_path = Path(vault) / subdirectory
    notes_in_subdirectory = subdirectory_path.glob(f'**/*.md') 
    relative_paths = [Path(note_path).relative_to(subdirectory_path)
                      for note_path in notes_in_subdirectory]
    vn_objects = [VaultNote(vault, rel_path=Path(subdirectory) / rel_path)
                  for rel_path in relative_paths]
    return {vn.name: main_of_notation(vn) for vn in vn_objects
            if note_is_of_type(vn, PersonalNoteTypeEnum.NOTATION_NOTE)}

The `notations_and_main_notes` function returns all the notation notes in a subdirectory of a vault

In [ ]:
vault = _test_directory() / 'test_vault_7'
sample_output = notations_and_main_notes(vault, subdirectory='')
print(sample_output)
assert 'foag_notation_M_n_bullet' in sample_output
assert 'some_reference_name_notation_O_X_this_file_has_no_links' in sample_output
assert sample_output['some_reference_name_notation_O_X_this_file_has_no_links'] is None
assert 'poonen_curves_notation_Z_X_T' in sample_output

{'foag_notation_M_n_bullet': 'twist_of_a_graded_module', 'some_reference_name_notation_k_t_formal_power_series_ring': 'some_note', 'some_reference_name_notation_O_X_this_file_has_no_links': None, 'some_reference_name_notation_Pic_C': 'divisor_class_group_of_a_curve', 'some_reference_name_notation_Spec_A': 'spectrum_of_a_ring', 'foag_notation_O_n': 'foag_15.2.1', 'foag_notation_O_text_Proj__S__n': 'foag_15.2.1', 'poonen_curves_notation_zeta_X_s_zeta_function_of_variety': 'poonen_curves_3.4.1 DEFINITION', 'poonen_curves_notation_Z_X_T': 'poonen_curves_3.4.1 DEFINITION', 'some_reference_name_notation_B_R': 'note_with_some_excessive_notation_notes', 'some_reference_name_notation_B_R_1': 'note_with_some_excessive_notation_notes', 'some_reference_name_notation_Jac_C': 'note_with_some_excessive_notation_notes', 'foag_notation__otimes_A_quad_obj_Mod_A_times_obj_Mod_A_longarrow_obj_Mod_A_': 'foag_1.3.5'}


Here is an example with a subdirectory specified:

In [ ]:
vault = _test_directory() / 'test_vault_7'
sample_output = notations_and_main_notes(vault, subdirectory='some_other_folder')
print(sample_output)
assert 'foag_notation_M_n_bullet' not in sample_output
assert 'some_reference_name_notation_O_X_this_file_has_no_links' not in sample_output
assert 'poonen_curves_notation_Z_X_T' in sample_output

{'foag_notation_O_n': 'foag_15.2.1', 'foag_notation_O_text_Proj__S__n': 'foag_15.2.1', 'poonen_curves_notation_zeta_X_s_zeta_function_of_variety': 'poonen_curves_3.4.1 DEFINITION', 'poonen_curves_notation_Z_X_T': 'poonen_curves_3.4.1 DEFINITION', 'some_reference_name_notation_B_R': 'note_with_some_excessive_notation_notes', 'some_reference_name_notation_B_R_1': 'note_with_some_excessive_notation_notes', 'some_reference_name_notation_Jac_C': 'note_with_some_excessive_notation_notes'}


Alternatively, we can specify a subdirectory by a `VaultNote` object; the directory that the `VaultNote` object is the subdirectory:

In [ ]:
vault = _test_directory() / 'test_vault_7'
vn = VaultNote(vault, name='galois_group')
sample_output = notations_and_main_notes(vault, subdirectory=None, note=vn)
print(sample_output)
assert 'foag_notation_M_n_bullet' in sample_output
assert 'some_reference_name_notation_O_X_this_file_has_no_links' in sample_output
assert sample_output['some_reference_name_notation_O_X_this_file_has_no_links'] is None
assert 'poonen_curves_notation_Z_X_T' not in sample_output

{'foag_notation_M_n_bullet': 'twist_of_a_graded_module', 'some_reference_name_notation_k_t_formal_power_series_ring': 'some_note', 'some_reference_name_notation_O_X_this_file_has_no_links': None, 'some_reference_name_notation_Pic_C': 'divisor_class_group_of_a_curve', 'some_reference_name_notation_Spec_A': 'spectrum_of_a_ring'}


## Check if there is a notation link in the `See Also` section of a standard information note.

In [ ]:
#| export
def notation_note_is_linked_in_see_also_section(
        notation_note: VaultNote,
        info_note: Optional[VaultNote] = None # The note in which to find the link to `notation_note`. Defaults to `None`, in which case the main note is determined to be the first linked note of `notation_note`.
        ) -> bool:
    """Return `True` if a notation note is linked in the `See Also`
    section of a standard information note. 
    """
    if not info_note:
        info_note = main_of_notation(notation_note, as_note=True)
    notes = notation_notes_linked_in_see_also_section(
        info_note, vault=notation_note.vault, as_vault_notes=False)
    return notation_note.name in notes


`notation_note_is_linked_in_see_also_section` Returns `True` if a notation note is linked in the `See Also` section of an information note:

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Pic_C')
info_note = VaultNote(vault, name='divisor_class_group_of_a_curve')
assert not notation_note_is_linked_in_see_also_section(notation_note, info_note)

notation_note = VaultNote(vault, name='foag_notation_M_n_bullet')
info_note = VaultNote(vault, name='twist_of_a_graded_module')
assert notation_note_is_linked_in_see_also_section(notation_note, info_note)

If `info_note` is not specified or if `info_note=None`, then the info note in question is the first note which the notation note links to, see the `main_of_notation` function:

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Pic_C')
assert not notation_note_is_linked_in_see_also_section(notation_note)

notation_note = VaultNote(vault, name='foag_notation_M_n_bullet')
assert notation_note_is_linked_in_see_also_section(notation_note)

## Add notation links to the `See Also` section

#### Add links to individual notation notes

In [ ]:
#| export
def add_notation_note_to_see_also(
        notation_note: VaultNote,
        info_note: Optional[VaultNote] = None, # The note in which to link `notation_note`. Defaults to `None`, in which case the main note is determined to be the first linked note of `notation_note`.
        do_not_repeat: bool = True # If `True`, do not add a link to `notation_note` in if there is already a such a link.
        ) -> None:
    """Add a link to a notation note in the `See Also` section of
    a standard information note.

    **Raises**

    - NoteDoesNotExistError
        - If the information note to link to does not exist.
    
    """
    if not info_note:
        info_note = main_of_notation(notation_note, as_note=True)
    if not info_note.exists():
        raise NoteDoesNotExistError
    if do_not_repeat and notation_note_is_linked_in_see_also_section(
            notation_note, info_note):
        return
    mf = MarkdownFile.from_vault_note(info_note)
    link = ObsidianLink(False, notation_note.name, 0, 0)
    mf.add_line_in_section(
        'See Also', {'type': MarkdownLineEnum.UNORDERED_LIST,
                     'line': f'- {str(link)}'})
    mf.write(info_note)



The `add_notation_note_to_see_also` method adds a link to the specified notation note to the specified standard information note.

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note = VaultNote(temp_vault, name='some_reference_name_notation_Pic_C')
    info_note = VaultNote(temp_vault, name='divisor_class_group_of_a_curve')
    assert not notation_note_is_linked_in_see_also_section(notation_note, info_note)    
    print("The note's text before adding the link:")
    print(info_note.text(), '\n\n')

    add_notation_note_to_see_also(notation_note, info_note)
    assert notation_note_is_linked_in_see_also_section(notation_note, info_note)
    print("The note's text after adding the link:")
    print(info_note.text())

The note's text before adding the link:
---
cssclass: clean-embeds
aliases: []
tags: [_meta/literature_note, _meta/definition, _meta/notation]
---
# Divisor class group of a curve[^1]
Let $C/k$ be a curve.

The **divisor class group** of $C$, denoted **$\operatorname{Pic} C$**, is defined as $\operatorname{Div} C / \operatorname{Princ} C$.  

# See Also

# Meta
## References

## Citations and Footnotes
[^1]: Citation
 


The note's text after adding the link:
---
cssclass: clean-embeds
aliases: []
tags: [_meta/literature_note, _meta/definition, _meta/notation]
---
# Divisor class group of a curve[^1]
Let $C/k$ be a curve.

The **divisor class group** of $C$, denoted **$\operatorname{Pic} C$**, is defined as $\operatorname{Div} C / \operatorname{Princ} C$.  

# See Also
- [[some_reference_name_notation_Pic_C]]

# Meta
## References

## Citations and Footnotes
[^1]: Citation



If `info_note` is not specified or is `None`, then the information note to add the link is determined to be the first note that `notation_note` links to:

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note = VaultNote(temp_vault, name='some_reference_name_notation_Pic_C')
    assert not notation_note_is_linked_in_see_also_section(notation_note)
    add_notation_note_to_see_also(notation_note)
    assert notation_note_is_linked_in_see_also_section(notation_note)

If the information note to add the link does not exist, then a `NoteDoesNotExistError` is raised:

If `do_not_repeat` is not specified or is `True`, then the link to the notation note is added only if such a link is not already present: 

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note = VaultNote(temp_vault, name='foag_notation_M_n_bullet')
    info_note = VaultNote(temp_vault, name='twist_of_a_graded_module')
    assert notation_note_is_linked_in_see_also_section(notation_note, info_note)
    original_text = info_note.text()
    add_notation_note_to_see_also(notation_note, do_not_repeat=True)
    new_text = info_note.text()
    assert original_text == new_text
    assert notation_note_is_linked_in_see_also_section(notation_note, info_note)

    # notation_note = VaultNote(temp_vault, name='some_reference_name_notation_Pic_C')
    # add_notation_note_to_see_also(notation_note)
    # assert notation_note_is_linked_in_see_also_section(notation_note)

Otherwise, the link to the notation note is added even if such a link is already present:

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note = VaultNote(temp_vault, name='foag_notation_M_n_bullet')
    info_note = VaultNote(temp_vault, name='twist_of_a_graded_module')
    original_text = info_note.text()
    add_notation_note_to_see_also(notation_note, do_not_repeat=False)
    new_text = info_note.text()
    assert original_text != new_text
    assert notation_note_is_linked_in_see_also_section(notation_note, info_note)
    print(new_text)


---
cssclass: clean-embeds
aliases: []
tags: [_meta/literature_note]
---
# Twist of graded module[^1]

Suppose $M_\bullet$ is a graded $S_\bullet$-module. Define the graded module **$M(n)_\bullet$** by $M(n)_{m}:=M_{n+m}$. Thus the quasicoherent sheaf $M(n)_\bullet$ satisfies

$$ \Gamma\left(D(f), \widetilde{M(n)}_{\bullet}\right)=\left(\left(M_{\bullet}\right)_{f}\right)_{n} $$

where here the subscript means we take the nth graded piece.

# See Also
- [[foag_notation_M_n_bullet]]
- [[foag_notation_M_n_bullet]]
# Meta
## References
![[_reference_foag]]

## Citations and Footnotes
[^1]: Vakil, Invertible 15.2, Page 412



If a link to the notation note is not already present, then `add_notation_note_to_see_also` adds such a link to the info note whether or not `do_not_repeat` is `True`:

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note = VaultNote(temp_vault, name='some_reference_name_notation_Pic_C')
    info_note = VaultNote(temp_vault, name='divisor_class_group_of_a_curve')
    assert not notation_note_is_linked_in_see_also_section(notation_note, info_note)    

    add_notation_note_to_see_also(notation_note, info_note, do_not_repeat=False)
    assert notation_note_is_linked_in_see_also_section(notation_note, info_note)

#### Add links to notation notes for all notation notes in a specified subdirectory.

In [ ]:
#| export
def add_missing_notation_links_to_information_notes(
        vault: PathLike, # Path to the vault directory.
        subdirectory: Optional[PathLike] = None, # Path to the subdirectory, relative to `vault`, to find the notation notes and their main notes. Searches for all notation notes here and in subdirectories of this subdirectory. If `None`, then the `note` parameter is used to determine `subdirectory`. Defaults to `None`. 
        note: Optional[VaultNote] = None # A note in the vault. The directory that this note is in determines the `subdirectory` parameter if it is `None`.  Defaults to `None`, in which case `subdirectory` must be specified.
        ) -> None:
    # TODO: deal with possibility that note does not exist.
    """For each notation note in a specified subdirectory, Add links
    to notation notes in their main information notes if the notation
    links are not already present.
    
    **Raises**

    - ValueError
        - If `subdirectory` and `note` are both `None`.
    """
    mains_dict = notations_and_main_notes(vault, subdirectory, note)
    to_check = {key: value for key, value in mains_dict.items()
                if value is not None}
    for notation, main in to_check.items():
        if not main:
            continue
        notation_note = VaultNote(vault, name=notation)
        try:
            main_note = VaultNote(vault, name=main)  # TODO add the subdirectory parameter here appropriately. See Also `notations_and_main_note` 
            add_notation_note_to_see_also(notation_note, main_note)
        except NoteDoesNotExistError:
            continue

The `add_missing_notation_links_to_information_notes` method adds links to notation notes of a specified directory to their main notes if these links are not already present. Note that nonexisting information notes which are the "main" notes of notation notes are ignored.

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note_1 = VaultNote(temp_vault, name='some_reference_name_notation_Pic_C')
    main_note_1 = VaultNote(temp_vault, name='divisor_class_group_of_a_curve')
    notation_note_2 = VaultNote(temp_vault, name='some_reference_name_notation_Spec_A')
    main_note_2 = VaultNote(temp_vault, name='spectrum_of_a_ring')
    notation_note_3 = VaultNote(temp_vault, name='foag_notation_O_n')
    main_note_3 = VaultNote(temp_vault, name='foag_15.2.1')

    assert not notation_note_is_linked_in_see_also_section(notation_note_1, main_note_1)
    assert not notation_note_is_linked_in_see_also_section(notation_note_2, main_note_2)
    assert not notation_note_is_linked_in_see_also_section(notation_note_3, main_note_3)
    add_missing_notation_links_to_information_notes(temp_vault, '')
    assert notation_note_is_linked_in_see_also_section(notation_note_1, main_note_1)
    assert notation_note_is_linked_in_see_also_section(notation_note_2, main_note_2)
    assert notation_note_is_linked_in_see_also_section(notation_note_3, main_note_3)

# Add notation notes to Notation index note

In [ ]:
#| export
def notations_to_add_in_index(
        vault: PathLike, # Path to the vault directory.
        notation_index_note = VaultNote, # The notation index note in the vault where the notations should be added to.
        subdirectory: Optional[PathLike] = None , # Path to the subdirectory, relative to `vault`, to find the notation notes. Searches for all notation notes here and in subdirectories of this subdirectory. If `None`, then the `note parameter is used to determined the subdirectory. If `subdirectory` is the empty str, then all notation notes in the vault are searched. Defaults to `None`. 
        note: Optional[VaultNote] = None # The directory that this note is in determines the argument to `subdirectory` parameter if it is `None`. Defaults to `None`, in which case `subdirectory` must be specified.
        ) -> list[tuple[str, ObsidianLink]]: # Each tuple in the list consists of the notation str of the notation note (including surrounding dollar signs `$`) and the (nonembedded) ObsidianLink object for a link to the notation note.
    """Returns notations and links of notation notes to that ought to be
    added in the corresponding notation index, i.e. are in the reference
    folder but not linked by the notation index note.

    If a notation note is not properly formatted, e.g. does not have a
    notation, then the notation and link for the notation note will not
    be included.
    
    **Raises**
    - ValueError
        - If `subdirectory` and `note` are both `None`.

    """
    vault = vault if vault is not None else ''
    mains_dict = notations_and_main_notes(vault, subdirectory, note)
    mf_object = MarkdownFile.from_file(notation_index_note.path())
    mf_text = str(mf_object)
    notations_and_links = []
    for notation, _ in mains_dict.items():
        link_object = ObsidianLink(
            is_embedded=True, file_name=notation, anchor=0, custom_text=0,
            link_type=LinkType.WIKILINK)
        link = link_object.to_string()
        try:
            notation_str = notation_in_note(notation, vault)
        except AttributeError:  # When a notation note is incomplete.
            continue  # TODO: print a warning
        if not link in mf_text:
            notations_and_links.append((notation_str, link_object))
    return notations_and_links



We can identify notation notes which exist in a reference folder but are not linked in the notation index note for the reference:

In [ ]:
# TODO: make test
# VaultNote.clear_cache()
vault = _test_directory() / 'test_vault_6'
reference = 'number_theory_reference_1'
note = VaultNote(vault, name=f'_index_{reference}')
notation_index_note = VaultNote(vault, name=f'_notation_{reference}')
sample_output = notations_to_add_in_index(vault, notation_index_note, note=note)
for notation_str, link in sample_output:
    print(notation_str, link.to_string())

$\mathbb{Z}/n\mathbb{Z}$ ![[number_theory_reference_1_notation_Z_nZ_ring_of_integers_modulo_n]]


In [ ]:
#| export
def index_notation_note_formatted_entry(
        notation_str: str, # The str of the notation, including the surrounding dollar signs `$`.
        link: ObsidianLink # The embedded link to the notation note. 
        ) -> str:
    """Return a str formatted for an index notation note entry.

    It is recommended to pass the outputs of
    `notations_to_add_in_index` to this function.
    """
    return f'### {notation_str}\n- {link.to_string()}'

The `index_notation_note_formatted_entry` function returns a formatted str to add in the index notation note:

In [ ]:
print(index_notation_note_formatted_entry(sample_output[0][0], sample_output[0][1]))

### $\mathbb{Z}/n\mathbb{Z}$
- ![[number_theory_reference_1_notation_Z_nZ_ring_of_integers_modulo_n]]


# Making a notation note

In [ ]:
#| export
def make_a_notation_note(
        main_note: VaultNote, # The note from which the notation originates.
        vault: PathLike,
        notation: str, # The notation typed in latex. May or may not be surrounded by dollar signs
        description: str, # The rest of the text describing notation.
        notation_note_name: str, # The name of the new notation note to be created.
        destination: Optional[PathLike] = None, # The directory to create the new notation note in.  If `None`, then creates the new notation note in the same place as the note specified by `note_name`
        overwrite: bool = False, # If `True`, overwrite file of the same path as the new notation file to be written, if such a file exists.  Otherwise, does nothing. Even if a link to the old notation note exists in `main_note`, a new link will still be added.  Defaults to `False`.
        add_to_main: bool = True # If `True`, adds a link to the notation note in the `See Also` section of the main note.
        ) -> Union[VaultNote, None]: # The newly created notation note. If no note is created, then returns `None`.
    """Make a new notation note, optionally add a link to it in the
    `See Also` section of its main note, returns it.

    The notation note is created in the same directory as the main note.
    The meta of the notation note has a `latex_in_original` section which
    lists the contents of the latex string in the main note from which the
    notation note comes from. This is so that the
    `make_notation_notes_from_double_asts` method can distinguish between
    notations for which a note has been created and for which a note has
    not been created.
    """
    if destination is None:
        destination = main_note.directory(relative=True)
    notation_note = VaultNote(
        vault, rel_path=destination / f'{notation_note_name}.md')
    if not overwrite and notation_note.exists():
        return
    if not notation_note.exists():
        notation_note.create()
    to_print = _full_notation_string(main_note, notation, description)
    # TODO: change this to use VaultNote method
    with open(notation_note.path(), 'w+', encoding='utf8') as notation_file:
        notation_file.write(to_print)
    if add_to_main:
        add_notation_note_to_see_also(notation_note, main_note)
    return notation_note
    

def _full_notation_string(
        main_note: VaultNote, notation: str, description: str) -> str:
    """The full "statement" of a notation.
    
    Says something like "<notation> denotes <description of notation>", e.g.
    "$\dim V$ denotes the dimension of the vector space $V$".
    
    **Parameters**
    - notation - str
        - Notation written in LaTeX.
    - description - str
        - The full description of the notation.
        
    **Returns**
    - str
    """
    raw_notation = f'{_raw_notation(notation)}'
    denote_link = ObsidianLink(False, main_note.name, 0, 'denotes')
    meta_notation = raw_notation.replace('\\', '\\\\')
    return (f'---\ndetect_regex: []\n'
            f'latex_in_original: ["{meta_notation}"]'
            f'\n---\n${raw_notation}$ {str(denote_link)} {description}')


def _raw_notation(notation: str):
    """
    """
    notation = notation.strip()
    notation = notation.strip('$')
    notation = notation.strip()
    return notation


We can make a notation note with the `make_a_notation_note` method.

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    note = VaultNote(temp_vault, name='galois_group')
    notation_note = make_a_notation_note(
        note, temp_vault, r'\operatorname{Gal}(L/K)', '', notation_note_name='some_reference_name_notation_Gal_L_K_galois_group')
    mf = MarkdownFile.from_vault_note(notation_note)
    assert mf.has_metadata()
    meta = mf.metadata()
    assert 'detect_regex' in meta
    assert 'latex_in_original' in meta
    print(meta, '\n')
    assert '\\operatorname{Gal}(L/K)' in meta['latex_in_original']
    print(mf, '\n')

    main_mf = MarkdownFile.from_vault_note(note)
    # print(main_mf)
    assert notation_note.name in str(main_mf)  # A link has been created
    # os.startfile(temp_vault)
    # input()

{'detect_regex': [], 'latex_in_original': ['\\operatorname{Gal}(L/K)']} 

---
detect_regex: []
latex_in_original: ["\\operatorname{Gal}(L/K)"]
---
$\operatorname{Gal}(L/K)$ [[galois_group|denotes]]  



Note that the surrounding dollar signs for LaTeX math mode can be included in the argument for `notation`:

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    note = VaultNote(temp_vault, name='galois_group')
    notation_note = make_a_notation_note(
        note, temp_vault, r'$\operatorname{Gal}(L/K)$', '', notation_note_name='some_reference_name_notation_Gal_L_K_galois_group')
    mf = MarkdownFile.from_vault_note(notation_note)
    assert mf.has_metadata()
    meta = mf.metadata()
    assert 'detect_regex' in meta
    assert 'latex_in_original' in meta
    print(meta, '\n')
    assert '\\operatorname{Gal}(L/K)' in meta['latex_in_original']
    print(mf, '\n')

    main_mf = MarkdownFile.from_vault_note(note)
    # print(main_mf)
    assert notation_note.name in str(main_mf)  # A link has been created
    # os.startfile(temp_vault)
    # input()

{'detect_regex': [], 'latex_in_original': ['\\operatorname{Gal}(L/K)']} 

---
detect_regex: []
latex_in_original: ["\\operatorname{Gal}(L/K)"]
---
$\operatorname{Gal}(L/K)$ [[galois_group|denotes]]  



Setting `add_to_main=False` only creates the notation note, but does not add a link to the notation note in the main note:

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    note = VaultNote(temp_vault, name='galois_group')
    notation_note = make_a_notation_note(
        note, temp_vault, r'\operatorname{Gal}(L/K)', '', notation_note_name='_reference_notation_Gal_L_K_galois_group',
        add_to_main=False)

    mf = MarkdownFile.from_vault_note(notation_note)
    assert mf.has_metadata()
    meta = mf.metadata()
    assert 'detect_regex' in meta
    assert 'latex_in_original' in meta
    print(meta, '\n')
    assert '\\operatorname{Gal}(L/K)' in meta['latex_in_original']
    print(mf, '\n')

    main_mf = MarkdownFile.from_vault_note(note)
    # print(main_mf)
    assert notation_note.name not in str(main_mf)  # No link has been created

{'detect_regex': [], 'latex_in_original': ['\\operatorname{Gal}(L/K)']} 

---
detect_regex: []
latex_in_original: ["\\operatorname{Gal}(L/K)"]
---
$\operatorname{Gal}(L/K)$ [[galois_group|denotes]]  



If the notation note of the specified name (`notation_note_name`) already exists, then by default no note is created and no link is added in the main note. 

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    note = VaultNote(temp_vault, name='spectrum_of_a_ring')
    notation_note = make_a_notation_note(
        note, temp_vault, r'\operatorname{Spec} A', '', notation_note_name='some_reference_name_notation_Spec_A')

    assert notation_note is None

    main_mf = MarkdownFile.from_vault_note(note)
    # print(main_mf)
    assert main_mf.get_headings_and_text()['# See Also'].strip() == ''  # No link has been added

Setting `overwrite=True`, however, will overwrite the existing note. The method will also add a link to the (overwritten) notation note.

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    note = VaultNote(temp_vault, name='spectrum_of_a_ring')
    notation_note = make_a_notation_note(
        note, temp_vault, r'\operatorname{Spec} A', '', notation_note_name='some_reference_name_notation_Spec_A',
        overwrite=True)

    assert notation_note is not None

    main_mf = MarkdownFile.from_vault_note(note)
    assert notation_note.name in str(main_mf)
    notation_mf = MarkdownFile.from_vault_note(notation_note) 
    # notation_mf has been overwritten
    print(notation_mf)

---
detect_regex: []
latex_in_original: ["\\operatorname{Spec} A"]
---
$\operatorname{Spec} A$ [[spectrum_of_a_ring|denotes]] 


In [ ]:
#| hide
vault = _test_directory() / 'test_vault_7'
note = VaultNote(vault, name='galois_group')
output = _full_notation_string(note, notation=r'\operatorname{Gal}(L/K)', description='')
print(output)
assert r'$\operatorname{Gal}(L/K)$' in output
assert '[[galois_group|denotes]]' in output

---
detect_regex: []
latex_in_original: ["\\operatorname{Gal}(L/K)"]
---
$\operatorname{Gal}(L/K)$ [[galois_group|denotes]] 


In [ ]:
#| hide
assert _raw_notation(r'$$$\mathscr{O}_X$$') == r'\mathscr{O}_X'
assert _raw_notation(r'$$ \operatorname{Spec} A  $$') == r'\operatorname{Spec} A'
assert _raw_notation(r'$f_\mathfrak{p}$') == r'f_\mathfrak{p}'
assert _raw_notation(r'$ \mathscr{O}_X') == r'\mathscr{O}_X'
assert _raw_notation(r'\operatorname{Gal}(L/K)') == r'\operatorname{Gal}(L/K)'

In [ ]:
#| hide
output_1 = _full_notation_string(note, notation=r'$$$\mathscr{O}_X$$', description='')
output_2 = _full_notation_string(note, notation=r'$$ \mathscr{O}_X $$', description='')
output_3 = _full_notation_string(note, notation=r'$\mathscr{O}_X$', description='')
output_4 = _full_notation_string(note, notation=r'$ \mathscr{O}_X', description='')
assert output_1 == output_2
assert output_2 == output_3
assert output_3 == output_4

In [ ]:
#| export
def make_notation_notes_from_double_asts(
        main_note: VaultNote, # The standard information note from which the notations are marked with double asterisks
        vault: PathLike, # The name of the reference; the notation note's name will start with `{reference_name}_notation_`.
        reference_name: str,
        destination: Optional[PathLike] = None, # The directory to create the new notation notes in.  If `None`, then creates the new notation note in the same place as the note specified by `note_name`
        overwrite: bool = False, # If `True`, overwrite file of the same path as the new notation file to be written, if such a file exists.  Otherwise, does nothing. Defaults to `False`.
        add_to_main: bool = True # If `True`, adds links to the notation note in the `See Also` section of the main note.
        ) -> list[VaultNote]: # The list of VaultNotes that are newly created/modified.
    """Make notation notes based on double asterisks surrounding LaTeX text in
    a standard information note.

    Notations are deemed to be completely LaTeX text in info notes that
    are surrounded by double asterisks. In basicality, if such a LaTeX
    text (without surrounding dollars signs `$` or `$$`) is listed in
    the `latex_in_original` metadata section of some notation note in the same
    directory as the info note whose main note is the info note in question,
    then a new notation note for that LaTeX text 
    is not created. However, if there are multiple instances of the same
    LaTeX text, then some notation notes may be created so that the number
    of times the LaTeX text appears in the info note is the no more than
    the number of times the LaTeX text appears in `latex_in_original` metadata
    sections of notation notes (in the same directory as the info note whose
    main note is the info note).

    For example, if there is an info note with notations `A`, `A`, `'A'`,
    `'A'`, and `B` and if there is a single notation note in the same
    directory as the info note with two `'A'` and `'A'` entries in its
    `latex_in_original` metadata section, then three notation notes will be
    created: two with `'A'` listed in their `latex_in_original` sections, and
    one with `'B'` listed in its `latex_in_original` section.

    **Raises**

    - Warning
        - If there are notation notes whose main note is determined to
        be to `main_note` and whose notations "excessively cover" those
        in `main_note`, i.e. the notation notes have more notations than
        `main_note` introduces. The main note and the excessive
        notations are printed; the notations are printed instead of the 
        notation notes because the same notation may span either multiple
        or single notation notes.
    """
    # Find notations
    notations = notation_str_in_a_standard_information_note(main_note)
    notations = [_raw_notation(notation) for notation in notations]
    # Get only the notations not already made into notes based on
    # latex_in_original
    all_latex_in_original = _latex_in_original_from_notat_notes_to_main_note(
        vault, main_note)
    notations_to_create = Multiset(notations).difference(all_latex_in_original)
    notations_to_create = list(notations_to_create)
    # Alert of existing notations that should not be there
    excess_notations = all_latex_in_original.difference(Multiset(notations))
    excess_notations = list(excess_notations)
    if excess_notations:
        warnings.warn(
            f"The following note has the following excess notations: "
            f"{main_note.name}, {', '.join(excess_notations)}")
    # Make notations
    return _make_new_notes_from_sifted_double_asts(
        main_note, vault, reference_name, notations_to_create,
        destination, overwrite, add_to_main)
    

def _latex_in_original_in_notat(
        notation_note: VaultNote
        ) -> list[str]:
    """Return the `latex_in_original` metadata section of the notation note.
    
    If the `latex_in_original` metadata section does not exist, then returns
    the list consisting of the notation in the notation note.
    """
    # TODO: test in the case that `latex_in_original` section does not exist
    mf = MarkdownFile.from_vault_note(notation_note)
    metadata = mf.metadata()
    if metadata is not None:
        return metadata.get('latex_in_original',
                            [notation_in_note(notation_note).strip('$')])
    else:
        return [notation_in_note(notation_note)]

    
def _latex_in_original_from_notat_notes_to_main_note(
        vault: PathLike,
        main_note: VaultNote # The info note
        ) -> Multiset:
    """Return a Multiset enumerating the entries of `latex_in_original`
    in the notation notes in the same directory as an info note
    """
    notation_notes_in_folder = notations_and_main_notes(vault, note=main_note)
    notation_notes_of_main_note = [
        VaultNote(vault, name=notation_note) for notation_note, info_note
        in notation_notes_in_folder.items()
        if main_note.name == info_note]

    all_latex_in_original = Multiset()
    for notat_note in notation_notes_of_main_note:
        all_latex_in_original.update(_latex_in_original_in_notat(notat_note))
    return all_latex_in_original


MAX_NOTE_NAME_LENGTH = 80
def _make_new_notes_from_sifted_double_asts(
        main_note: VaultNote, vault: PathLike, reference_name: str,
        notations: list[str], destination: Optional[PathLike],
        overwrite: bool, add_to_main: bool) -> list[VaultNote]:
    """
    Actually makes the notation notes found from the double asterisked LaTeX
    str's
    """
    # TODO: test that note names aren't too long.
    new_notes = []
    for notation in reversed(notations):
        notation_note_name = f'{reference_name}_notation_'\
            f'{latex_to_path_accepted_string(notation)}'
        if len(notation_note_name) > MAX_NOTE_NAME_LENGTH:
            notation_note_name = notation_note_name[:MAX_NOTE_NAME_LENGTH]
        notation_note_name = VaultNote.unique_name(
            notation_note_name, vault)
        new_note = make_a_notation_note(
            main_note, vault, notation, '', notation_note_name,
            destination, overwrite, add_to_main)
        if new_note:
            new_notes.append(new_note)
    return new_notes
    



As described in `markdown.obsidian.personal.machine_learning.notation_identification`, we surround a LaTeX math mode string with double asterisks `**` to indicate that the string introduces a notation.

The `make_notation_notes_from_double_asts` method parses LaTeX surrounded by double asterisks `**` in a standard information note and automatically creates notation notes for said LaTeX.

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_6'
    shutil.copytree(_test_directory() / 'test_vault_6', temp_vault)

    # os.startfile(temp_vault)
    # input()
    info_note = VaultNote(temp_vault, name='reference_for_notation_notes_introducing_some_notations')
    new_notes = make_notation_notes_from_double_asts(info_note, temp_vault, 'reference_for_notation_notes')
    
    assert len(new_notes) == 3
    for new_note in new_notes:
        assert new_note.exists()
        assert notation_note_is_linked_in_see_also_section(new_note, info_note)
    # input()
    # TODO: add more tests - overwrite=True, add_to_main=False

In the following example, we prompt `make_notation_notes_from_double_asts` to make notation notes for an info note with no notations - nothing is modified:

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_6'
    shutil.copytree(_test_directory() / 'test_vault_6', temp_vault)

    info_note = VaultNote(temp_vault, name='reference_for_notation_notes_no_notations_introduced_here')
    info_note_content_before = info_note.text()
    new_notes = make_notation_notes_from_double_asts(info_note, temp_vault, 'reference_for_notation_notes')
    info_note_content_after = info_note.text()
    
    assert len(new_notes) == 0
    assert info_note_content_before == info_note_content_after

In the following example, we prompt `make_notation_notes_from_double_asts` on the same info note twice - no new notation notes are created the second time.

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_6'
    shutil.copytree(_test_directory() / 'test_vault_6', temp_vault)

    info_note = VaultNote(temp_vault, name='reference_for_notation_notes_introducing_some_notations')
    make_notation_notes_from_double_asts(info_note, temp_vault, 'milne_av')
    new_notes = make_notation_notes_from_double_asts(info_note, temp_vault, 'milne_av')
    assert len(new_notes) == 0
    

In the following example, an info note contains two of the same notation. One notation note for each of these notations is created, but with different names:

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    info_note = VaultNote(temp_vault, name='note_with_repeated_notation')
    new_notes = make_notation_notes_from_double_asts(info_note, temp_vault, 'some_reference_name')
    assert len(new_notes) == 2
    print(new_notes[0].name)
    print(new_notes[1].name)

some_reference_name_notation_Cl_K
some_reference_name_notation_Cl_K_1


In the following example, there are notation notes with the info note as their main note, but some of the notations in these notation notes cover those in the info note "excessively" - in this case, only notation notes to uncovered notations are created, and warnings are raised to indicate which notations are covered excessively.

In [ ]:
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    info_note = VaultNote(temp_vault, name='note_with_some_excessive_notation_notes')
    new_notes = make_notation_notes_from_double_asts(info_note, temp_vault, 'some_reference_name')
    assert len(new_notes) == 1

C:\Users\hyunj\AppData\Local\Temp\ipykernel_7972\3896227388.py:57: UserWarning: The following note has the following excess notations: note_with_some_excessive_notation_notes, \mathcal{B}(\mathbb{R}), \operatorname{Jac}(C)
  warnings.warn(


In [ ]:
#| hide
with tempfile.TemporaryDirectory(prefix='tmp_dir_', dir=os.getcwd()) as tmp_dir:
    tmp_dir = Path(tmp_dir)
    temp_vault = tmp_dir / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    info_note = VaultNote(temp_vault, name='foag_1.3.5')
    new_notes = make_notation_notes_from_double_asts(info_note, temp_vault, 'foag')
    assert len(new_notes) == 0

In [ ]:
# TODO: move notation notes to directory of main notes

# Decomposition of notation

In [ ]:
# #| export
# def decompose_notation_as_sequence(
#         notation # latex styled. Assumed to not be surrounded with `'$'`.
#         ) -> list:
#     """
#     **Parameters**
#     - notation - str
    
#     **Returns**
#     - list of str
#     """
#     str_index = 0
#     decomposition = []
#     while str_index < len(notation):
#         if notation[str_index] == '\\':
#             j = 1
#             while (str_index + j < len(notation)
#                     and notation[str_index + j] not in [' ', '\\', '{', '(']):
#                 j += 1
#             latex_command = notation[str_index:str_index + j]
#             decomposition.append(latex_command)
#             str_index += j
#         if str_index < len(notation):
#             decomposition.append(notation[str_index])
#         str_index += 1
#     return decomposition


# def compare_notations_for_sorting(
#         notation1, # latex styled. Assumed to not be surrounded with `'$'`.
#         notation2, # latex styled. Assumed to not be surrounded with `'$'`.
#         character_ordering_list
#         ) -> int:
#     """
#     **Parameters**
#     - notation - str
#         - latex styled. Assumed to not be covered with `'$'`.
    
#     **Returns**
#     - int
#         - 1 if `notation2` is considered to come "earlier", -1 if
#         `notation1` is considered to come "earlier", and 0 otherwise.
#     """
#     decomposition1 = decompose_notation_as_sequence(notation1)
#     decomposition2 = decompose_notation_as_sequence(notation2)
#     index1, index2 = 0, 0
#     while index1 < len(decomposition1) and index2 < len(decomposition2):
#         return  #TODO
#     if index1 < len(decomposition1):
#         return 1
#     if index2 < len(decomposition2):
#         return -1
#     return 0


# def _find_next_effective_character(decomposition, index):
#     while (index < len(decomposition)
#             and decomposition[index] in ['{', '}', '(', ')', '[', ']', r'\tilde',
#                                          r'\hat', r'\bar', 'r\overline']):
#         return

In [ ]:
# decompose_notation_as_sequence(r'\tilde{K} (X)')
# decompose_notation_as_sequence(r'P(X;*,A)')
# decompose_notation_as_sequence(r'\operatorname{Spec} A')

## Detect notations being used in reference

### Regex from latex

In [ ]:
#| export
SPECIAL_CHARACTERS = ['.', '+', '*', '?', '^', '$', '(', ')',
                      '[', ']', '{', '}', '|', '\\']
replaceable_groups = [['mathrm', 'operatorname', 'rm', 'text'],
                      ['mathbf', 'bf'],
                      ['mathit', 'it']]


def _build_replacables_from_groups(
        replaceable_groups: list[list[str]]) -> dict[str, set[str]]:
    total_dict = {}
    for listy in replaceable_groups:
        set_for_group = set(listy)
        for macro in listy:
            total_dict[macro] = set_for_group
    return total_dict


REPLACEABLES = _build_replacables_from_groups(replaceable_groups)
    

def regex_from_latex(
        latex: str, replaceables: dict[str, set[str]] = REPLACEABLES,
        special_characters: list[str] = SPECIAL_CHARACTERS) -> str:
    """Returns regex to match latex math mode string which is essentially
    equivalent to a specified latex math mode string.
    
    The outputs of this function may not work correctly.
    The regex pattern does not have to fully match equivalent string.
    
    **Parameters**

    - latex - str
        - The latex math mode string. Does not include math mode delimiters
        such as `$`, `$$`, `\[ \]` (although the characters `'\['` and `'\]'`
        can still be part of the string, e.g. for optional arguments of a
        macro/operator). Can include "placeholders" `r'\1'`, `r'\2'`, `r'\3'`,
        etc. to indicate substitutable/generics; the placeholders can be
        substituted with any string.
    - replaceables - dict[str, set[str]]
        - latex strings/commands which are considered "interreplacable"
    - special_characters - list[str]
        - characters to add a backslash `'\'` in front of for regex.
        Defaults to a list consisting of special characters in regex.
    """
    if not replaceables:
        replaceables = {}
    w = LatexWalker(latex)
    nodelist, _, _ = w.get_latex_nodes(pos=0)
    regex_parts = []
    # print(nodelist)
    for node in nodelist:
        _look_into_node(node, regex_parts,
                        replaceables, special_characters)
    regex_parts.append('(?:[ \\{\\}]*)')
    return ''.join(regex_parts)
    
def _look_into_node(
        node: LatexNode, regex_parts: list[str],
        replaceables: dict[str, set[str]],
        special_characters: list[str]) -> None:
    """Appends to `regex_parts`"""
    # hasattr(node, 'nodeargd')
    # print(node)
    if isinstance(node, LatexMacroNode):
        macroname = node.macroname
        if _macro_is_actually_placeholder(macroname):
            regex_parts.append('(?:.*)')
        else:
            if macroname in replaceables:
                replaceable_macros = replaceables[macroname]
            else:
                replaceable_macros = [macroname]
            options_str = '|'.join(replaceable_macros)
            options_str = f'(?:{options_str})'
            regex_parts.append(fr'(?: *?)\\{options_str}(?: *?)')
        for node in node.nodeargd.argnlist:
            _look_into_node(node, regex_parts,
                            replaceables, special_characters)
    elif isinstance(node, LatexGroupNode):
        # print('\nGroup Node')
        # print(node)
        # print(node.nodelist)
        delimiters = node.delimiters
        regex_parts.append(f'\\{delimiters[0]}(?: *?)')
        for node in node.nodelist:
            _look_into_node(node, regex_parts,
                            replaceables, special_characters)
        regex_parts.append(f'(?: *?)\\{delimiters[1]}')
    elif isinstance(node, LatexCharsNode):
        # print('\nChars Node')
        # print(node)
        # print(node.chars)
        chars = node.chars.strip()
        chars = list(chars)
        chars = [f'\\{char}' if char in special_characters else char
                 for char in chars]
        # print(chars)
        chars.insert(0, '')  # add the misc spaces/brackets front and back
        chars.append('')
        regex_optional_spaces_and_brackets = '(?:[ \\{\\}]*?)'.join(chars)
        regex_parts.append(regex_optional_spaces_and_brackets)
        
def _macro_is_actually_placeholder(macro: str) -> bool:
    return macro.isnumeric()

In [ ]:
text = r"""e"""
print(regex_from_latex(text, REPLACEABLES))

(?:[ \{\}]*?)e(?:[ \{\}]*?)(?:[ \{\}]*)


### Get regex from notation note

So far, I have just made notation notes in the form `'$math_mode_string$ denotes ...'`. I want to add frontmatter metadata in notation notes to indicate regex to detect the notation with placeholders.

In [ ]:
#| export
def regex_from_notation_note(vault: PathLike, note: VaultNote) -> str:
    """Returns a regex str to detect the notation of the notation note.
    
    The regex detection strings should be in a list labeled `detect_regex` in
    the yaml frontmatter. If multiple strings are in the list, then the regex
    will detect latex math mode strings roughly corresponding to any of them.
    If multiple strings are in the list, then they must be ordered 
    "by priority", with the higher priority regexes coming first. It is good
    to have these string in quotes `""` to make sure that yaml can load them
    safely. When doing so, make sure to escape characters, e.g. backslash
    should be typed as `\\`, etc.
    
    The strings in `detect_regex` can include placeholders, cf.
    ``regex_from_latex``.
    
    **Parameters**
    - vault - PathLike
    - note - VaultNote
    
    **Returns**
    - str
        - Of the regex used to detect the notation. The regex does not need to
        fully match instances of the notation.
    """
    assert note_is_of_type(note, PersonalNoteTypeEnum.NOTATION_NOTE)
    mf = MarkdownFile.from_vault_note(note)
    metadata = mf.metadata()
    if metadata and 'detect_regex' in metadata:
        detects = metadata['detect_regex']
        regexes = [regex_from_latex(detect) for detect in detects]
        return '|'.join(regexes)
    else:
        notation = notation_in_note(note, vault)
        return regex_from_latex(notation[1:-1])  # Get rid of `'$'`.

In [ ]:
# TODO: test